In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter
import numpy as np
import dask.dataframe as dd

In [ ]:
# Caricare il dataset
df3 = pd.read_csv('df_tipo1.csv')

#print(df3)
#df3 = df3.drop(columns = ['Unnamed: 0'])

df3.replace({7: -1, 9: -1, 77 : -1, 99 : -1, 777: -1, 999: -1, 7777: -1, 9999: -1 ,'nan': -1}, inplace=True)
df3.replace(-1.0, np.nan, inplace=True)
# Contare i NaN per colonna
nan_counts = df3.isna().sum()
#nan_count_col1 = df3['DIABTYPE'].isna().sum()
#print(f'Numero di NaN in col1: {nan_count_col1}')
print(df3.info())

In [ ]:
# Determinare la soglia
soglia = 120000  # Modificare questo valore in base alla soglia desiderata

# Se il numero di valori mancanti in una colonna è maggiore della soglia, eliminare la colonna
df_cleaned = df3.drop(columns=nan_counts[nan_counts > soglia].index)
df_cleaned = df_cleaned.dropna(thresh=df_cleaned.shape[1] - 0)
df_sampled = df_cleaned.astype({col: 'float32' for col in df_cleaned.select_dtypes(include='float64').columns})

#df_sampled = df_cleaned.replace(np.nan, -1.0)
print(df_sampled.info())
print(df_sampled)
#df_sampled = df_sampled.drop(columns = ['SMOKE100','INCOME3','_AGEG5YR.1', 'EDUCA'])
#rint(df_sampled.dtypes)

#df3 = df3.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
#df3 = df3.fillna(-1)

In [ ]:
# Dividere il dataset in train e test
train_data, test_data = train_test_split(df_sampled, test_size=0.2, random_state=42)

In [ ]:
# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df_sampled)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(df_sampled), max_indegree = 2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 #   print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")

In [ ]:
features = df_sampled.columns.tolist()
model_nodes = set(model.nodes())
presenti = [feature for feature in features if feature in model_nodes]
non_presenti = [feature for feature in features if feature not in model_nodes]

print(presenti)
print(non_presenti)

In [ ]:
# Reset indici
df_test_data = test_data.reset_index(drop=True)

In [ ]:
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Previsione sul test set
y_true = df_test_data['DIABTYPE1'].values
y_pred = []

df_test_data.drop(columns=['DIABTYPE1'], inplace=True)

# Funzione per salvare le previsioni su file
def salva_previsioni(y_pred, blocco):
    print(f'Saving: {blocco}')
    with open(f'previsioni_blocco_{blocco}.txt', 'w') as file:
        for pred in y_pred:
            file.write(f"{pred}\n")

In [ ]:
stati_validi = {}
for node in model.nodes():
    cpd = model.get_cpds(node)
    print(node)
    print(cpd.state_names[node])
    stati_validi[node] = cpd.state_names[node]

In [ ]:
len(stati_validi['WEIGHT2'])

In [ ]:
valuetof = 1367.3837890625
if valuetof in df_sampled['_LLCPWT2'].values:
    print("presente")
    

In [ ]:
# Iterare il dataframe a blocchi di 100 righe
blocco = 0
for start in range(blocco*100, len(df_test_data), 100):
    end = start + 100
    df_blocco = train_data.iloc[start:end]
    print(f'Prediction per: {start}:{end}, blocco {blocco}')

    
    for index, row in df_blocco.iterrows():
        print(f'Prediction for interrows: {index}, {start}:{end}, blocco {blocco}')

        evidence = row.to_dict()
        evidence.pop('SLEPTIM1')
        #for node, value in evidence.items():
            #cpd = model.get_cpds(node)
            #valid_states = cpd.state_names[node]
            #if value not in valid_states:
                #raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        #filtered_evidence = {k: v for k, v in evidence.items() if k in model_nodes}
        #print(filtered_evidence)
        
        try:
            prediction = inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
            
            # Debug: stampa il risultato della previsione
            print(f'Prediction per index {index}: {prediction}')
            
            # Verifica il tipo di prediction
            if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                y_pred.append(prediction['DIABTYPE1'])
            else:
                print(f'Predizione non valida per index {index}: {prediction}')
                y_pred.append(None)
                
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    
    # Salvare le previsioni su file dopo ogni blocco di 100 righe
    salva_previsioni(y_pred, blocco)
    blocco += 1
    y_pred = []  # Reset della lista per il prossimo blocco

In [ ]:

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
